In [5]:
import pandas as pd
from tqdm import tqdm
import csv

# === Fonction pour détecter le séparateur ===
def detect_separator(file_path, encoding="utf-8-sig"):
    with open(file_path, 'r', encoding=encoding) as f:
        sample = f.read(2048)
        sniffer = csv.Sniffer()
        dialect = sniffer.sniff(sample)
        return dialect.delimiter

# === Chemins des fichiers ===
base_path = "C:/Users/MSI/Desktop/Attijari_bank/stage/"
fact_path = base_path + "fact_operations.csv"
dim_clients_path = base_path + "dim_clients.csv"
dim_comptes_path = base_path + "dim_comptes.csv"
dim_date_path = base_path + "dim_date.csv"
dim_type_op_path = base_path + "dim_type_operation.csv"

# === Détection et chargement de tous les fichiers CSV ===
print("📥 Chargement des fichiers avec détection de séparateurs...")

sep_clients = detect_separator(dim_clients_path)
sep_comptes = detect_separator(dim_comptes_path)
sep_date = detect_separator(dim_date_path)
sep_type_op = detect_separator(dim_type_op_path)

print(f"   ↪ sep_clients      = '{sep_clients}'")
print(f"   ↪ sep_comptes     = '{sep_comptes}'")
print(f"   ↪ sep_date        = '{sep_date}'")
print(f"   ↪ sep_type_op     = '{sep_type_op}'")

# === Chargement des fichiers ===
dim_clients = pd.read_csv(dim_clients_path, sep=sep_clients, dtype={"id_client": str})
dim_comptes = pd.read_csv(dim_comptes_path, sep=sep_comptes, dtype={"id_client": str, "id_compte": str})
dim_date = pd.read_csv(dim_date_path, sep=sep_date, dtype={"id_date": str})
dim_type_op = pd.read_csv(dim_type_op_path, sep=sep_type_op, dtype={"id_type_operation": int})

# === Vérification colonnes ===
required_cols = ['id_type_operation', 'type_operation']
if not all(col in dim_type_op.columns for col in required_cols):
    raise ValueError(f"❌ Colonnes requises manquantes dans dim_type_operation : {dim_type_op.columns.tolist()}")

# === Dictionnaire id_type_operation → nom ===
type_op_dict = dim_type_op.set_index('id_type_operation')['type_operation'].to_dict()

# === Configuration chunks ===
chunk_size = 100000
total_lines = sum(1 for _ in open(fact_path, 'r', encoding='utf-8-sig')) - 1
estimated_chunks = total_lines // chunk_size + 1

# === Lecture en chunks avec Sniffer + filtrage 'cheque' ===
fact_cheques_chunks = []
print(f"\n📊 Lecture en chunks ({chunk_size} lignes) - Total estimé : {estimated_chunks} chunks")

with pd.read_csv(fact_path, dtype={"id_compte": str, "id_date": str, "id_type_operation": int}, 
                 chunksize=chunk_size, encoding="utf-8-sig") as reader:
    for i, chunk in enumerate(tqdm(reader, total=estimated_chunks, desc="⏳ Progression chunks")):
        chunk['type_operation'] = chunk['id_type_operation'].map(type_op_dict)
        filtered = chunk[
            (chunk['type_operation'].str.lower().str.contains("cheque", na=False)) &
            (chunk['sens_operation'] == "DEBIT")
        ]
        fact_cheques_chunks.append(filtered)

# === Fusion des chunks ===
fact_cheques = pd.concat(fact_cheques_chunks, ignore_index=True)
print(f"✅ Total opérations chèque : {len(fact_cheques):,}")

# === Jointure dim_date ===
fact_cheques = fact_cheques.merge(dim_date[['id_date', 'annee', 'mois']], on='id_date', how='left')
nan_dates = fact_cheques['annee'].isna().sum()
if nan_dates > 0:
    print(f"⚠️ {nan_dates:,} lignes sans année supprimées")
    fact_cheques = fact_cheques.dropna(subset=['annee'])

# === Jointure dim_comptes → id_client + ancienneté ===
# MODIFICATION: Ajout des colonnes anciennete et tranche_anciennete
fact_cheques = fact_cheques.merge(
    dim_comptes[['id_compte', 'id_client', 'anciennete_compte', 'tranche_anciennete']], 
    on='id_compte', 
    how='left'
)
nan_comptes = fact_cheques['id_client'].isna().sum()
if nan_comptes > 0:
    print(f"⚠️ {nan_comptes:,} lignes sans client supprimées")
    fact_cheques = fact_cheques.dropna(subset=['id_client'])

# === Jointure dim_clients ===
fact_cheques = fact_cheques.merge(dim_clients[['id_client', 'type_client', 'salaire', 'emploi']], on='id_client', how='left')
nan_clients = fact_cheques['type_client'].isna().sum()
if nan_clients > 0:
    print(f"⚠️ {nan_clients:,} lignes sans profil supprimées")
    fact_cheques = fact_cheques.dropna(subset=['type_client'])

# === Agrégation annuelle ===
agg_annuel = fact_cheques.groupby(['id_client', 'annee']).agg({
    'nombre_cheques': 'sum',
    'montant_total': 'sum'
}).rename(columns={
    'nombre_cheques': 'nb_cheques',
    'montant_total': 'montant_cheques'
}).reset_index()

pivot_nb = agg_annuel.pivot(index='id_client', columns='annee', values='nb_cheques').fillna(0)
pivot_montant = agg_annuel.pivot(index='id_client', columns='annee', values='montant_cheques').fillna(0)
pivot_nb.columns = [f'nb_cheques_{int(c)}' for c in pivot_nb.columns]
pivot_montant.columns = [f'montant_cheques_{int(c)}' for c in pivot_montant.columns]
features_cheques = pivot_nb.join(pivot_montant).reset_index()

# === Moyennes et tendance ===
features_cheques['nb_cheques_annuel_moyen'] = features_cheques[['nb_cheques_2022', 'nb_cheques_2023', 'nb_cheques_2024']].mean(axis=1)
features_cheques['montant_cheques_annuel_moyen'] = features_cheques[['montant_cheques_2022', 'montant_cheques_2023', 'montant_cheques_2024']].mean(axis=1)

def detecter_tendance(row):
    v1, v2, v3 = row['nb_cheques_2022'], row['nb_cheques_2023'], row['nb_cheques_2024']
    if v3 > v2 > v1:
        return 'croissante'
    elif v3 < v2 < v1:
        return 'décroissante'
    return 'stable'

features_cheques['tendance_evolution'] = features_cheques.apply(detecter_tendance, axis=1)

# === Ratio chèques / montant total ===
reader_total = pd.read_csv(fact_path, dtype={"id_compte": str, "id_date": str}, chunksize=chunk_size, encoding="utf-8-sig")
montant_total_dict = {}

for chunk in tqdm(reader_total, total=estimated_chunks, desc="🔄 Calcul montant total"):
    chunk = chunk.merge(dim_comptes[['id_compte', 'id_client']], on='id_compte', how='left')
    agg = chunk.groupby('id_client')['montant_total'].sum()
    for id_client, montant in agg.items():
        montant_total_dict[id_client] = montant_total_dict.get(id_client, 0) + montant

df_total = pd.DataFrame.from_dict(montant_total_dict, orient='index', columns=['montant_total_ops']).reset_index().rename(columns={'index': 'id_client'})
features_cheques = features_cheques.merge(df_total, on='id_client', how='left')
features_cheques['ratio_cheques_vs_total'] = features_cheques['montant_cheques_2024'] / features_cheques['montant_total_ops']
features_cheques['ratio_cheques_vs_total'] = features_cheques['ratio_cheques_vs_total'].fillna(0).round(4)

# === Ajout profil client final ===
profil = dim_clients[['id_client', 'type_client', 'salaire', 'emploi']]
features_cheques = features_cheques.merge(profil, on='id_client', how='left')
nan_final = features_cheques['type_client'].isna().sum()
if nan_final > 0:
    print(f"⚠️ {nan_final:,} lignes sans profil supprimées")
    features_cheques = features_cheques.dropna(subset=['type_client'])

# === Ajout du plafond_cheque basé sur le salaire ===
print("\n📊 Calcul du plafond_cheque basé sur le salaire...")

# Détection clients avec au moins 1 compte éligible au chéquier
eligibilite = dim_comptes.groupby('id_client')['eligible_chequier'].any().reset_index()
eligibilite['eligible_chequier'] = eligibilite['eligible_chequier'].fillna(False)

# Joindre avec salaire depuis dim_clients
profil_salaire = dim_clients[['id_client', 'salaire']].merge(eligibilite, on='id_client', how='left')
profil_salaire['eligible_chequier'] = profil_salaire['eligible_chequier'].fillna(False)

# Formule logique plafond
def calculer_plafond_cheque(row):
    salaire = row['salaire']
    eligible = row['eligible_chequier']
    if not eligible:
        return 0.0
    if salaire < 1000:
        return salaire * 1.0
    elif salaire < 3000:
        return salaire * 1.5
    elif salaire < 7000:
        return salaire * 2.0
    else:
        return salaire * 3.0

profil_salaire['plafond_cheque'] = profil_salaire.apply(calculer_plafond_cheque, axis=1).round(2)

# === Fusion avec table des features ===
features_cheques = features_cheques.merge(profil_salaire[['id_client', 'plafond_cheque']], on='id_client', how='left')
features_cheques['plafond_cheque'] = features_cheques['plafond_cheque'].fillna(0)

# Optionnel : Ajouter tranche_plafond
features_cheques['tranche_plafond'] = pd.cut(
    features_cheques['plafond_cheque'],
    bins=[0, 1000, 5000, 10000, float('inf')],
    labels=['0-1k', '1k-5k', '5k-10k', '10k+']
)

# === NOUVELLE SECTION: Ajout des données d'ancienneté par client ===
print("\n📊 Ajout des informations d'ancienneté...")

# Pour chaque client, on prend l'ancienneté maximale de ses comptes
# (ou on peut faire une moyenne selon votre logique métier)
anciennete_client = fact_cheques.groupby('id_client').agg({
    'anciennete_compte': 'max',  # ou 'mean' pour la moyenne
    'tranche_anciennete': lambda x: x.mode().iloc[0] if not x.empty else None  # valeur la plus fréquente
}).reset_index()

# Fusion avec la table des features
features_cheques = features_cheques.merge(anciennete_client, on='id_client', how='left')

# Gestion des valeurs manquantes si nécessaire
features_cheques['anciennete_compte'] = features_cheques['anciennete_compte'].fillna(0)
features_cheques['tranche_anciennete'] = features_cheques['tranche_anciennete'].fillna('Non définie')

# === Export final avec plafond ET ancienneté ===
output = base_path + "features_cheques_utilisateurs.csv"
features_cheques.to_csv(output, index=False, encoding='utf-8-sig')
print(f"✅ Export avec plafond_cheque ET ancienneté terminé : {output}")
print(f"📊 Clients avec plafond > 0 : {(features_cheques['plafond_cheque'] > 0).sum():,}")
print(f"📊 Clients avec ancienneté > 0 : {(features_cheques['anciennete_compte'] > 0).sum():,}")
print(f"\n✅ Export terminé : {output}")
print(f"📊 Clients utilisateurs finaux : {len(features_cheques):,}") 

# === Affichage des statistiques d'ancienneté ===
print("\n📈 Statistiques d'ancienneté:")
print(features_cheques['anciennete_compte'].describe())
print(f"\n📊 Répartition par tranche d'ancienneté:")
print(features_cheques['tranche_anciennete'].value_counts())

📥 Chargement des fichiers avec détection de séparateurs...
   ↪ sep_clients      = ','
   ↪ sep_comptes     = ','
   ↪ sep_date        = ','
   ↪ sep_type_op     = ';'

📊 Lecture en chunks (100000 lignes) - Total estimé : 675 chunks


⏳ Progression chunks: 100%|██████████| 675/675 [02:02<00:00,  5.53it/s]


✅ Total opérations chèque : 30,706,363


🔄 Calcul montant total: 100%|██████████| 675/675 [02:28<00:00,  4.56it/s]



📊 Calcul du plafond_cheque basé sur le salaire...

📊 Ajout des informations d'ancienneté...
✅ Export avec plafond_cheque ET ancienneté terminé : C:/Users/MSI/Desktop/Attijari_bank/stage/features_cheques_utilisateurs.csv
📊 Clients avec plafond > 0 : 155,210
📊 Clients avec ancienneté > 0 : 155,210

✅ Export terminé : C:/Users/MSI/Desktop/Attijari_bank/stage/features_cheques_utilisateurs.csv
📊 Clients utilisateurs finaux : 155,210

📈 Statistiques d'ancienneté:
count    155210.000000
mean       2586.727073
std         897.358678
min         580.000000
25%        1909.000000
50%        2739.000000
75%        3361.000000
max        3866.000000
Name: anciennete_compte, dtype: float64

📊 Répartition par tranche d'ancienneté:
tranche_anciennete
6+ ans     80584
3-6 ans    51251
1-3 ans    23375
Name: count, dtype: int64


In [6]:
# Vérifie les doublons de id_client
duplicated_clients = features_cheques['id_client'].duplicated()

# Compter le nombre de doublons
nb_duplicated = duplicated_clients.sum()

if nb_duplicated > 0:
    print(f"❌ {nb_duplicated} doublons trouvés dans id_client")
else:
    print("✅ Aucun doublon : id_client est unique")


✅ Aucun doublon : id_client est unique


In [7]:
nb_eligibles = eligibilite['eligible_chequier'].sum()
print(f"📊 Clients éligibles au chéquier : {nb_eligibles:,}")


📊 Clients éligibles au chéquier : 155,210


In [8]:
# Filtrer dim_comptes pour récupérer les id_client éligibles
id_clients_eligibles_dim_comptes = set(
    dim_comptes[dim_comptes['eligible_chequier'] == True]['id_client']
)
print(f"📊 Clients éligibles dans dim_comptes : {len(id_clients_eligibles_dim_comptes):,}")
# Extraire les id_client du tableau final
id_clients_final = set(features_cheques['id_client'])
print(f"📊 Clients dans le tableau final : {len(id_clients_final):,}")
# Clients éligibles mais absents dans le tableau final
eligibles_non_inclus = id_clients_eligibles_dim_comptes - id_clients_final

if eligibles_non_inclus:
    print(f"❌ {len(eligibles_non_inclus):,} clients éligibles dans dim_comptes ne sont PAS dans le tableau final.")
    print(f"🔍 Exemples : {list(eligibles_non_inclus)[:5]}")
else:
    print("✅ Tous les clients éligibles dans dim_comptes sont présents dans le tableau final.")


📊 Clients éligibles dans dim_comptes : 155,210
📊 Clients dans le tableau final : 155,210
✅ Tous les clients éligibles dans dim_comptes sont présents dans le tableau final.


In [9]:
print("\n📊 Taille des tables chargées :")
print(f"   ➤ dim_clients        : {len(dim_clients):,} lignes")
print(f"   ➤ dim_comptes        : {len(dim_comptes):,} lignes")
print(f"   ➤ dim_date           : {len(dim_date):,} lignes")
print(f"   ➤ dim_type_operation : {len(dim_type_op):,} lignes")



📊 Taille des tables chargées :
   ➤ dim_clients        : 180,000 lignes
   ➤ dim_comptes        : 359,723 lignes
   ➤ dim_date           : 1,096 lignes
   ➤ dim_type_operation : 33 lignes


In [10]:
total_lines = sum(1 for _ in open(fact_path, 'r', encoding='utf-8-sig')) - 1
print(f"   ➤ fact_operations    : {total_lines:,} lignes")


   ➤ fact_operations    : 67,411,391 lignes


In [18]:
features_cheques.head()

,id_client,nb_cheques_2022,nb_cheques_2023,nb_cheques_2024,montant_cheques_2022,montant_cheques_2023,montant_cheques_2024,nb_cheques_annuel_moyen,montant_cheques_annuel_moyen,tendance_evolution,montant_total_ops,ratio_cheques_vs_total,type_client,salaire,emploi,plafond_cheque,tranche_plafond,anciennete_compte,tranche_anciennete
0,1000054,110.0,44.0,85.0,4914.28,1886.43,2787.52,79.666667,3196.076667,stable,24433.34,0.1141,Particulier,1233,Ouvrier specialise,1849.5,1k-5k,2309,6+ ans
1,1000131,230.0,246.0,193.0,26758.25,28252.49,19437.17,223.000000,24815.970000,stable,198305.44,0.0980,Professionnel,2325,Infirmier,3487.5,1k-5k,2801,6+ ans
2,1000132,93.0,324.0,355.0,12388.26,38919.47,45600.66,257.333333,32302.796667,croissante,261054.87,0.1747,Professionnel,6360,Chef de projet,12720.0,10k+,2752,1-3 ans
3,1000134,179.0,312.0,289.0,26460.47,43131.58,31713.17,260.000000,33768.406667,stable,287267.62,0.1104,Professionnel,2557,Technico-commercial,3835.5,1k-5k,2999,3-6 ans
4,1000412,222.0,210.0,198.0,28462.06,25252.52,25631.09,210.000000,26448.556667,décroissante,221017.96,0.1160,Professionnel,3233,Technico-commercial,6466.0,5k-10k,3698,6+ ans
